## Wielomiany interpolujące

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sym
from sympy import latex
from sympy.printing import cxxcode

import re

from IPython.display import display, Latex

# Równanie dyfuzji nośników

Równanie do rozwiązania ma postać:
$$
D \Delta u - A u - B u^2 - C u^3 + J = 0,
$$
gdzie
$$
J = \frac{j_\perp}{q_e\,d},
$$
w którym $j_\perp$ to wstrzykiwana gęstość prądu a $d$ to grubość złącza. Po zamianie na równanie linowe (korzystając z rozwinięcia w szereg Taylora) mamy:
$$
D \Delta u - E u + F = 0,
$$
gdzie
\begin{align}
E & = A + 2 B u_0 + 3 C u_0^2, \\
F & = B u_0^2 + 2 C u_0^3 + J_0.
\end{align}

In [2]:
A, B, C, D, = sym.symbols('A B C D')

Zastosujmy metodę Ritza w we współrzędnych kartezjańskich

In [3]:
x, y = sym.symbols('x y')
X, Y = sym.symbols('X Y')

ξ = x / X
ζ = y / Y

Jako funkcje próbne Używamy wielomianów trzeciego rzędu, które pozwalają na zagwarantowanie ciągłości funkcji i pochodnych:

In [4]:
Φ = np.linalg.inv(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 1, 1, 1],
    [0, 1, 2, 3]
])).T

φx = [sum(int(c) * ξ**i for i,c in enumerate(p)) for p in Φ]
φx[1] *= X
φx[3] *= X
φx = np.array([sym.expand(a) for a in φx])

φy = [sum(int(c) * ζ**i for i,c in enumerate(p)) for p in Φ]
φy[1] *= Y
φy[3] *= Y
φy = np.array([sym.expand(a) for a in φy])

## Funkcja przybliżana

Zakładamy, że
$$
u(x,y) = \sum_{i,j} u^{i,j} \, \varphi_{i,j}(x,y) \qquad u_0(x,y) = \sum_{i,j} U^{i,j} \,\varphi_{i,j}(x,y)
$$
gdzie
\begin{align}
u^{0,0}, u^{1,0}, u^{0,1} & = u(0,0), \left.\tfrac{du}{dx}\right|_{0,0}, \left.\tfrac{du}{dy}\right|_{0,0} \\
u^{2,0}, u^{3,0}, u^{2,1} & = u(X,0), \left.\tfrac{du}{dx}\right|_{X,0}, \left.\tfrac{du}{dy}\right|_{X,0} \\
u^{0,2}, u^{1,2}, u^{0,3} & = u(0,Y), \left.\tfrac{du}{dx}\right|_{0,Y}, \left.\tfrac{du}{dy}\right|_{0,Y} \\
u^{2,2}, u^{3,2}, u^{2,3} & = u(X,Y), \left.\tfrac{du}{dx}\right|_{X,Y}, \left.\tfrac{du}{dy}\right|_{X,Y} \\
\end{align}


Interpolacja musi być zrobiona tak, była sumetryczna względem zamiany $x$ i $y$. W związku z tym musi ona mieć postać
\begin{alignat}{9}
u(x,y) & = \varphi_0(x)\,\varphi_0(y)\,u^{0,0} && + \varphi_1(x)\,\varphi_0(y)\,u^{1,0} && + \varphi_2(x)\,\varphi_0(y)\,u^{2,0} && + \varphi_3(x)\,\varphi_0(y)\,u^{3,0} \\
       & + \varphi_0(x)\,\varphi_1(y)\,u^{0,1} &&                                       && + \varphi_2(x)\,\varphi_1(y)\,u^{2,1} && \\
       & + \varphi_0(x)\,\varphi_2(y)\,u^{0,2} && + \varphi_1(x)\,\varphi_2(y)\,u^{1,2} && + \varphi_2(x)\,\varphi_2(y)\,u^{2,2} && + \varphi_3(x)\,\varphi_2(y)\,u^{3,2} \\
       & + \varphi_0(x)\,\varphi_3(y)\,u^{0,3} &&                                       && + \varphi_2(x)\,\varphi_3(y)\,u^{2,3}. &&
\end{alignat}

In [5]:
φ = np.array([
    [φx[0] * φy[0], φx[1] * φy[0], φx[2] * φy[0], φx[3] * φy[0]],
    [φx[0] * φy[1],             0, φx[2] * φy[1],             0],
    [φx[0] * φy[2], φx[1] * φy[2], φx[2] * φy[2], φx[3] * φy[2]],
    [φx[0] * φy[3],             0, φx[2] * φy[3],             0]
])

idx = [(0,0), (0,1), (0,2), (0,3), (1,0), (1,2), (2,0), (2,1), (2,2), (2,3), (3,0), (3,2)]
φ = np.array([φ[idx[i]] for i in range(12)])

In [6]:
dφx = np.array([sym.expand(sym.diff(a, x)) for a in φ])
dφy = np.array([sym.expand(sym.diff(a, x)) for a in φ])

In [7]:
U = sym.IndexedBase('U', shape=(4,4))
u0 = sum(U[idx[k]] * φ[k] for k in range(12))
u0

(3*x**2/X**2 - 2*x**3/X**3)*(3*y**2/Y**2 - 2*y**3/Y**3)*U[2, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(-y**2/Y + y**3/Y**2)*U[3, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(y - 2*y**2/Y + y**3/Y**2)*U[1, 2] + (-x**2/X + x**3/X**2)*(3*y**2/Y**2 - 2*y**3/Y**3)*U[2, 3] + (-x**2/X + x**3/X**2)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 3] + (3*y**2/Y**2 - 2*y**3/Y**3)*(1 - 3*x**2/X**2 + 2*x**3/X**3)*U[2, 0] + (3*y**2/Y**2 - 2*y**3/Y**3)*(x - 2*x**2/X + x**3/X**2)*U[2, 1] + (-y**2/Y + y**3/Y**2)*(1 - 3*x**2/X**2 + 2*x**3/X**3)*U[3, 0] + (1 - 3*x**2/X**2 + 2*x**3/X**3)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 0] + (1 - 3*x**2/X**2 + 2*x**3/X**3)*(y - 2*y**2/Y + y**3/Y**2)*U[1, 0] + (1 - 3*y**2/Y**2 + 2*y**3/Y**3)*(x - 2*x**2/X + x**3/X**2)*U[0, 1]

In [8]:
sym.collect(4 * u0.subs(x, X/2).subs(y, Y/2), (X/4,Y/4))

X*(U[0, 1] - U[0, 3] + U[2, 1] - U[2, 3])/4 + Y*(U[1, 0] + U[1, 2] - U[3, 0] - U[3, 2])/4 + U[0, 0] + U[0, 2] + U[2, 0] + U[2, 2]

In [9]:
try:
    from sympy.printing.cxx import CXX11CodePrinter
except ImportError:
    from sympy.printing.cxxcode import CXX11CodePrinter


def _print_Indexed(self, expr):
    indices = expr.indices
    if len(indices) == 1:
        # return f"{self._print(expr.base.label)}[idx(e,{self._print(indices[0])})]"
        return f"{self._print(expr.base.label)}{self._print(indices[0])}"
    else:
        # return f"{self._print(expr.base.label)}[idx(e,{','.join(self._print(i) for i in indices)})]"
        return f"{self._print(expr.base.label)}{''.join(self._print(i) for i in indices)}"
CXX11CodePrinter._print_Indexed = _print_Indexed

In [10]:
def cpp(v):
    s = cxxcode(sym.simplify(v), standard='C++11')
    #for i in range(1, 5):
    #    s = s.replace(f"a[{i}]", f"a{i}")
    s = re.sub(r'std::pow\(([^)]+), (\d+)\)', r'std::pow(\1,\2)', s)
    s = re.sub(r'std::pow\(([^)]+),2\)', r'\1*\1', s)
    s = re.sub(r'std::pow\(([^)]+),3\)', r'\1*\1*\1', s)
    s = re.sub(r'\b(X|Y)\b', r'e.\1', s)
    s = re.sub(r'\bU(\d\d)\b', r'U[e.i\1]', s)
    s = re.sub(r'\bJ(\d\d)\b', r'J[e.n\1]', s)
    s = re.sub(r'\bP(\d\d)(\d)\b', r'P[e.n\1].c\2\2', s)
    s = re.sub(r'(d?G)(\d)\b', r'\1.c\2\2', s)
    return s

In [12]:
print(cpp(u0))

(e.X*x*(-x*y*y*(e.X - x)*(3*e.Y - 2*y)*U[e.i23] - x*(e.X - x)*(e.Y*e.Y*e.Y - 3*e.Y*y*y + 2*y*y*y)*U[e.i03] + y*y*(3*e.Y - 2*y)*(e.X*e.X - 2*e.X*x + x*x)*U[e.i21] + (e.X*e.X - 2*e.X*x + x*x)*(e.Y*e.Y*e.Y - 3*e.Y*y*y + 2*y*y*y)*U[e.i01]) + e.Y*y*(-x*x*y*(3*e.X - 2*x)*(e.Y - y)*U[e.i32] + x*x*(3*e.X - 2*x)*(e.Y*e.Y - 2*e.Y*y + y*y)*U[e.i12] - y*(e.Y - y)*(e.X*e.X*e.X - 3*e.X*x*x + 2*x*x*x)*U[e.i30] + (e.X*e.X*e.X - 3*e.X*x*x + 2*x*x*x)*(e.Y*e.Y - 2*e.Y*y + y*y)*U[e.i10]) + x*x*y*y*(3*e.X - 2*x)*(3*e.Y - 2*y)*U[e.i22] + x*x*(3*e.X - 2*x)*(e.Y*e.Y*e.Y - 3*e.Y*y*y + 2*y*y*y)*U[e.i02] + y*y*(3*e.Y - 2*y)*(e.X*e.X*e.X - 3*e.X*x*x + 2*x*x*x)*U[e.i20] + (e.X*e.X*e.X - 3*e.X*x*x + 2*x*x*x)*(e.Y*e.Y*e.Y - 3*e.Y*y*y + 2*y*y*y)*U[e.i00])/(e.X*e.X*e.X*e.Y*e.Y*e.Y)


## Interpolacja prądu

Aby dobrze odzdwierciedlić rozpływ prądu, zakładamy, że na elemencie zmienia się on co najwyżej liniowo

In [ ]:
# z = np.array([L / 4, L * 3 / 4])
zx = np.array([0, X])
zy = np.array([0, Y])

Teraz zakładmy, że w tych punktach mamy wartości funkcji $J$. Budujemy wielomiany interpolacyjne.

In [ ]:
J = sym.IndexedBase('J', shape=(2,2))

def La(i, x, z):
    res = 1
    for j in range(len(z)):
        if j == i: continue
        res *= (x - z[j]) / (z[i] - z[j])
    return res

In [ ]:
Jxy = sym.simplify(sum(J[i,j] * La(i, x, zx) * La(j, y, zy) for i in range(len(zx)) for j in range(len(zy))))
Jxy

# Macierze sztywności i wektor obciążeń

Macierz sztywności budujemy z równania
$$
\left( - D \Delta + A + 2 B u_0 + 3 C u_0^2 \right) u = B u_0^2 + 2 C u_0^3 + J_0.
$$
Pamiętając, że
$$
u(x,y) = \sum_{i_x,i_y} u^{i_x,i_y} \, \varphi_{i_x,i_y}(x,y)
$$
i stosując metodę Ritza mamy
$$
\sum_{j_x,j_y} \int_0^L \left[ D \left( \frac{d\varphi_{i_x,i_y}}{dx} \frac{d\varphi_{j_x,j_y}}{dx} + \frac{d\varphi_{i_x,i_y}}{dy} \frac{d\varphi_{j_x,j_y}}{dy} \right) + \left( A + 2 B u_0 + 3 C u_0^2 \right) \varphi_{i_x,i_y} \varphi_{j_x,j_y} \right] u^j = \left( B u_0^2 + 2 C u_0^3 + J_0 \right) \varphi_{i_x,i_y}.
$$

Do tego, stosujemy
$$
u_0(x,y) = \sum_{i_x,i_y} U^{i_x,i_y} \,\varphi_{i_x,i_y}(x,y).
$$


In [ ]:
execfile('matrices2.py')

In [ ]:
K = KD + KA + KB + KC
K

In [ ]:
F = FB + FC + F0
F

# Wypalanie nośników

Wypalanie nośników w ogólności opisane jest wzorem
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{g(\mathbf{r}, u(\mathbf{r}))}{\hbar \omega} \frac{P\,M(\mathbf{r})}{1-R} = \frac{\lambda}{h\,c} \, g(\mathbf{r},u(\mathbf{r})) \, \frac{P\,M(\mathbf{r})}{1-R},
$$
gdzie $P$ to moc emitowana, $R$ odbijalność zwierciadeł, $M(\mathbf{r})$ to unormowany bezwymiarowy profil modu, $g(\mathbf{r},u(\mathbf{r}))$ wzmocnienie.

PLaSK dostarcza bezpośrednio natężenie modu w obszarza czynnym $P(\mathbf{r})$. Zatem powyższe równanie ma postać
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{\lambda}{h\,c} \, g(\mathbf{r},u(\mathbf{r})) \, P(\mathbf{r}).
$$

Zakładamy, że na pojedynczym elemencie natężenie zmienia się liniowo. Z kolei wzmocnienie jest liniową funkcją koncentracji i na całym elemencie ma postać
$$
g(\mathbf{r},u(\mathbf{r})) \approx g(u_0) + g_u \left(u(\mathbf{r}) - u_0 \right) = g(u_0) - g_u\,u_0 + g_u\,u(\mathbf{r}).
$$
Daje to
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{\lambda}{h\,c} \, P(\mathbf{r}) \, g_u\,u(\mathbf{r}) - \frac{\lambda}{h\,c} \, P(\mathbf{r}) \, \left[ g_u\,u_0 - g(u_0) \right].
$$

In [ ]:
P = sym.IndexedBase('P', shape=(2,2,2))
G = sym.IndexedBase('G', shape=(2,))
dG = sym.IndexedBase('dG', shape=(2,))
uG = sym.symbols('uG')


In [ ]:
Pxy = [sym.simplify(sum(P[i, j, k] * La(i, x, zx) * La(j, y, zy) for i in range(len(zx)) for j in range(len(zy)))) for k in range(2)]

In [ ]:
execfile('matrices2-shb.py')

In [ ]:
KL

In [ ]:
FL